In [ ]:
!pip install git+https://github.com/iguchi-lab/vtsim4/                  #vtsim4 vtsimcモジュール 
!pip install japanize-matplotlib                                        #日本語mtaplotlibモジュール

from vtsim import vtsim as vt
import japanize_matplotlib

# 温度差

In [ ]:
def calc1():

    input = {
        'index': vt.df(1).index,
        'sn': {
            'AC-out':           {'t_flag': vt.SN_CALC},
            'Room1':            {'t_flag': vt.SN_CALC},
            'Room2':            {'t_flag': vt.SN_CALC},
            'AC-in':            {'t_flag': vt.SN_FIX, 't': 20.0},
            'Outside':          {'t_flag': vt.SN_FIX, 't':  0.0},
            'solar':            {'h_sr':  400},
            'h_inp':            {'h_inp': 2000}
        },
        'vn': {
            'AC-in -> AC-out':  {'ac-vol': 1000 / 3600},
            'AC-out -> Room1':  {'vol':     500 / 3600},
            'AC-out -> Room2':  {'vol':     500 / 3600},
            'Room1 -> AC-in':   {'vol':     500 / 3600},
            'Room2 -> AC-in':   {'vol':     500 / 3600}
        },
        'tn': {
            'Room1 -> Outside': {'cdtc': 100 * 0.87},
            'Room2 -> Outside': {'cdtc': 200 * 0.87},
            'AC-in -> AC-out':  {'ac_mode': vt.AC_AUTO, 'pre_tmp': 20.0},
            'Room2 -> solar':   {'ms': 1.0}
        }
    }

    vt.write_json(input, 'tutorial_01.json')
    vt.run_calc(input)

calc1()

# 東大阪

In [ ]:
def calc2(CASE):
    from vtsim import ground as gnd
    from vtsim import fan as fan

    if   CASE == 0:     rg, case, df_i = 0.15, 'case0',       vt.read_csv('input_data_1h.csv') 
    elif CASE == 1:     rg, case, df_i = 1.85, 'case1_1h',    vt.read_csv('input_data_1h.csv') 
    elif CASE == 2:     rg, case, df_i = 0.15, 'case1_10min', vt.read_csv('input_data_10min.csv')
    elif CASE == 3:     rg, case, df_i = 0.15, 'case2_10min', vt.read_csv('input_data_10min.csv')
    elif CASE == 4:     rg, case, df_i = 0.15, 'case3_10min', vt.read_csv('input_data_10min.csv')

    area = {'A': 6.62, 'B': 22.36, 'C': 6.62, 'D': 7.45, 'E': 4.97}
    leng = {'A': 4.55, 'B':  9.10, 'C': 6.37, 'D': 6.37, 'E': 3.19}
    alpha, area_hatch, a_dash, n_value = 0.6, 0.6 * 0.35, 3 / 3600, 1.5
    u_value, phi_value = 2.31, 0.39
    vol  = df_i['hex_sts'].replace({0: 200 / 3600})

    fan_dict    = fan.fs.fan_dict(df_i['fan_sts'], {0: 'RARA0', 1: 'RARA1', 2: 'RARA2'})    
    ground_dict = {'rg': rg, **gnd.gs.ground_dict(case)}

    input = {
        'index': df_i.index,
        'sn': {
            '居室':    {'v_flag': vt.SN_FIX,  't_flag': vt.SN_FIX,  't': df_i['t_in']},
            'AC吹出':  {'v_flag': vt.SN_FIX,  't_flag': vt.SN_FIX,  't': df_i['t_supply']},
            'HEX吹出': {'v_flag': vt.SN_FIX,  't_flag': vt.SN_FIX,  't': (df_i['t_in'] - df_i['t_ex']) * 0.8 + df_i['t_ex']},
            '床下A':   {'v_flag': vt.SN_CALC, 't_flag': vt.SN_CALC},
            '床下B':   {'v_flag': vt.SN_CALC, 't_flag': vt.SN_CALC},
            '床下C':   {'v_flag': vt.SN_CALC, 't_flag': vt.SN_CALC},
            '床下D':   {'v_flag': vt.SN_CALC, 't_flag': vt.SN_CALC},
            '床下E':   {'v_flag': vt.SN_CALC, 't_flag': vt.SN_CALC},
            '外部':    {'t_flag': vt.SN_FIX,  't': df_i['t_ex']},
            '地盤':    {'t_flag': vt.SN_FIX,  't': df_i['t_ug']}
        },
        'vn': {
            '床下A -> 床下B':       {'alpha': alpha, 'area': area_hatch},
            '床下B -> 床下C':       {'alpha': alpha, 'area': area_hatch},
            '床下B -> 床下D':       {'alpha': alpha, 'area': area_hatch},
            '床下D -> 床下E':       {'alpha': alpha, 'area': area_hatch},
            '床下A -> 居室 : gap':  {'a': area['A'] * a_dash, 'n': n_value},
            '床下B -> 居室 : gap':  {'a': area['B'] * a_dash, 'n': n_value},
            '床下C -> 居室 : gap':  {'a': area['C'] * a_dash, 'n': n_value},
            '床下D -> 居室 : gap':  {'a': area['D'] * a_dash, 'n': n_value},
            '床下E -> 居室 : gap':  {'a': area['E'] * a_dash, 'n': n_value},
            '居室 -> AC吹出':       {'vol': df_i['Qv-AC'] / 3600},
            'AC吹出 -> 床下A':      {'vol': df_i['Qv-AC'] / 3600},
            '居室 -> HEX吹出':      {'vol': vol},
            'HEX吹出 -> 床下A':     {'vol': vol},
            '床下B -> 居室 : fan':  {**fan_dict},
            '床下C -> 居室 : fan':  {**fan_dict},
            '床下D -> 居室 : fan1': {**fan_dict},
            '床下D -> 居室 : fan2': {**fan_dict},
            '床下E -> 居室 : fan':  {**fan_dict}
        },
        'tn': {
            '床下A -> 居室': {'cdtc': area['A'] * u_value},
            '床下B -> 居室': {'cdtc': area['B'] * u_value},
            '床下C -> 居室': {'cdtc': area['C'] * u_value},
            '床下D -> 居室': {'cdtc': area['D'] * u_value},
            '床下E -> 居室': {'cdtc': area['E'] * u_value},
            '床下A -> 外部': {'cdtc': leng['A'] * phi_value},
            '床下B -> 外部': {'cdtc': leng['B'] * phi_value},
            '床下C -> 外部': {'cdtc': leng['C'] * phi_value},
            '床下D -> 外部': {'cdtc': leng['D'] * phi_value},
            '床下E -> 外部': {'cdtc': leng['E'] * phi_value},
            '床下A -> 地盤': {'area': area['A'], **ground_dict},
            '床下B -> 地盤': {'area': area['B'], **ground_dict},
            '床下C -> 地盤': {'area': area['C'], **ground_dict},
            '床下D -> 地盤': {'area': area['D'], **ground_dict},
            '床下E -> 地盤': {'area': area['E'], **ground_dict}
        }
    }

    vt.write_json(input, 'tutorial_osaka.json')
    vt.run_calc(input)

calc2(1)

# IBEC断熱

In [ ]:
def calc3():
    import itertools
    from datetime import datetime, timedelta

    length = 1000

    n_n   = lambda x, y, z: str(x) + '-' + str(y) + '-' + str(z)
    h_cz  = lambda z: sum(cz[:z]) + cz[z] / 2
    h_tz  = lambda z: sum(cz[:z + 1])
    cx, cy, cz  = [1.52, 1.52, 1.52], [1.02, 1.60, 1.02], [0.48, 0.48, 0.48, 0.48, 0.48]
    X, Y, Z =len(cx), len(cy), len(cz)
    cdtc1, cdtc2, cdtc3, cdtc4, cdtc5, cdtc6 = 0.4, 0.5, 0.94, 1.88, 3.49, 5.59

    sn, vn, tn = {}, {}, {}

    for x, y, z in itertools.product(range(X), range(Y), range(Z)):
        sn[n_n(x, y, z)] = {'v_flag': vt.SN_CALC, 't_flag': vt.SN_CALC, 't': 20.0, 'capa': cx[x] * cy[y] * cz[z] * 1.205 * 1006}

    sn['外部'] = {'t_flag': vt.SN_FIX, 't':  6.0}
    sn['隣室'] = {'t_flag': vt.SN_FIX, 't': 20.0}
    sn['発熱'] = {'h_inp': 400}

    for x, y, z in itertools.product(range(X), range(Y), range(Z)):    
        if x != X - 1:  vn[n_n(x, y, z) + ' -> ' + n_n(x + 1, y, z)] = {'alpha': 1.0, 'area': cy[y] * cz[z], 'h1': h_cz(z), 'h2': h_cz(z)}
        if y != Y - 1:  vn[n_n(x, y, z) + ' -> ' + n_n(x, y + 1, z)] = {'alpha': 1.0, 'area': cx[x] * cz[z], 'h1': h_cz(z), 'h2': h_cz(z)}
        if z != Z - 1:  vn[n_n(x, y, z) + ' -> ' + n_n(x, y, z + 1)] = {'alpha': 1.0, 'area': cx[x] * cy[y], 'h1': h_tz(z), 'h2': h_tz(z)}

        if (z == 0) or (z == Z - 1):    tn[n_n(x, y, z) + ' -> ' + '外部'] = {'cdtc': cdtc1 * cx[x] * cy[y]}
        if (y == 0) or (y == Y - 1):    tn[n_n(x, y, z) + ' -> ' + '外部'] = {'cdtc': cdtc1 * cx[x] * cz[z]}

        if x == 0:
            if(y == 1) and (z < 4):     tn[n_n(x ,y, z) + ' -> ' + '外部'] = {'cdtc': cdtc6 * cy[y] * cz[z]}
            else:                       tn[n_n(x, y, z) + ' -> ' + '外部'] = {'cdtc': cdtc1 * cy[y] * cz[z]}
        if x == X - 1:                  tn[n_n(x, y, z) + ' -> ' + '隣室'] = {'cdtc': cdtc1 * cy[y] * cz[z]}

    tn[n_n(2, 1, 0) + ' -> ' + '発熱'] = {'type': vt.TN_HEATER}

    input = {
        'calc_sts': {
            'solve': vt.SOLVE_SOR, 'vent_err': 1e-3
        },
        'index': vt.df(length).index, 'sn': sn, 'vn': vn, 'tn': tn
    }

    vt.write_json(input, 'tutorial_ibec.json')
    vt.run_calc(input)

calc3()

vt.df_qv.loc[::, ['0-1-0 -> 0-1-1', '0-1-1 -> 0-1-2', '0-1-2 -> 0-1-3', '0-1-3 -> 0-1-4',
                  '2-1-0 -> 2-1-1', '2-1-1 -> 2-2-1', '2-1-2 -> 2-1-3', '2-1-3 -> 2-1-4']].plot()
vt.df_t.loc[::, ['0-1-0', '0-1-1', '0-1-2', '0-1-3', '0-1-4', '2-1-0', '2-1-1', '2-1-2', '2-1-3', '2-1-4', '1-1-4']].plot()

# 浦和１

In [ ]:
def calc4(CASE):
    from vtsim import ground as gnd
    import pandas as pd
    import matplotlib.pyplot as plt

    FN_LIST = [['cooling-L.csv', 'm_cooling-L.csv'],
               ['cooling-M.csv', 'm_cooling-M.csv'],
               ['cooling-H.csv', 'm_cooling-H.csv'],
               ['heating-L.csv', 'm_heating-L.csv'],
               ['heating-M.csv', 'm_heating-M.csv'],
               ['heating-H.csv', 'm_heating-H.csv']]

    in_fn, ms_fn = FN_LIST[CASE]
    df_i = vt.read_csv(in_fn)

    ground_dict = {'rg': 0.15, **gnd.gs.ground_dict('case3_10min')}

    input = {'index':   df_i.index,
             'sn': {'建築ダクト': {'t_flag': vt.SN_FIX,  't':    df_i['t-supply']},
                    'LDK':        {'t_flag': vt.SN_CALC, 'capa': 12586.97 * 1000},
                    '洗面所':     {'t_flag': vt.SN_CALC, 'capa':  3923.02 * 1000},
                    '洋室(大)':   {'t_flag': vt.SN_CALC, 'capa':  7130.62 * 1000},
                    '洋室(小)':   {'t_flag': vt.SN_CALC, 'capa':  4491.65 * 1000},
                    'ホール':     {'t_flag': vt.SN_CALC, 'capa': 10976.77 * 1000},
                    '外部':       {'t_flag': vt.SN_FIX,  't':    df_i['t-ex']},
                    '地盤':       {'t_flag': vt.SN_FIX,  't':    16.0},
                    '日射':       {'t_flag': vt.SN_FIX,  'h_sr': df_i['h_sr']}
                   },
             'vn': {'建築ダクト -> LDK':      {'vol': 300 / 3600},
                    '建築ダクト -> 洗面所':   {'vol':  50 / 3600},
                    '建築ダクト -> 洋室(大)': {'vol': 250 / 3600},
                    '建築ダクト -> 洋室(小)': {'vol': 100 / 3600},
                    '建築ダクト -> ホール':   {'vol': 100 / 3600},
                    'LDK -> ホール':          {'vol': 300 / 3600},
                    '洗面所 -> ホール':       {'vol':  50 / 3600},
                    '洋室(大) -> ホール':     {'vol': 250 / 3600},
                    '洋室(小) -> ホール':     {'vol': 100 / 3600},
                    'ホール -> 建築ダクト':   {'vol': 800 / 3600}
                   }, 
             'tn': {'LDK -> 外部':      {'cdtc': 54.03},
                    '洗面所 -> 外部':   {'cdtc':  7.31},
                    '洋室(大) -> 外部': {'cdtc': 25.29},
                    '洋室(小) -> 外部': {'cdtc':  8.32},
                    'ホール -> 外部':   {'cdtc': 35.54},
                    'LDK - > 地盤':     {'area': 38.09, **ground_dict},
                    '洗面所 -> 地盤':   {'area':  4.64, **ground_dict},
                    'ホール -> 地盤':   {'area': 17.39, **ground_dict},
                    'LDK -> 日射':      {'ms':    1.61},
                    '洗面所 - >日射':   {'ms':    0.23},
                    '洋室(大) -> 日射': {'ms':    0.98},
                    '洋室(小) -> 日射': {'ms':    0.49},
                    'ホール ->日射':    {'ms':    0.99}
                   }
            }

    vt.write_json(input, 'tutorial_urawa_1.json')
    vt.run_calc(input)


    df_output = vt.df_t[8760::]
    df_measure = pd.read_csv(ms_fn, index_col = 0, parse_dates = True)    

    fig = plt.figure(facecolor='w', figsize = (15, 10))
    fig.subplots_adjust(wspace = -0.1, hspace=0.7)

    a = fig.add_subplot(5, 1, 1)
    a.plot(df_output['LDK'], linewidth = 1.0, label = 'calc-LDK')
    a.plot(df_measure['リビング'], linewidth = 1.0, label = 'リビング')
    a.legend(ncol=5, bbox_to_anchor=(0, 1.05, 1, 0), loc='lower left', borderaxespad=0, facecolor='w', edgecolor='k')

    a = fig.add_subplot(5, 1, 2)
    a.plot(df_output['洗面所'], linewidth = 1.0, label = 'calc-洗面所')
    a.plot(df_measure['洗面所'], linewidth = 1.0, label = '洗面所')
    a.legend(ncol=5, bbox_to_anchor=(0, 1.05, 1, 0), loc='lower left', borderaxespad=0, facecolor='w', edgecolor='k')

    a = fig.add_subplot(5, 1, 3)
    a.plot(df_output['洋室(大)'], linewidth = 1.0, label = 'calc-洋室(大)')
    a.plot(df_measure['洋室東'], linewidth = 1.0, label = '洋室東')
    a.legend(ncol=5, bbox_to_anchor=(0, 1.05, 1, 0), loc='lower left', borderaxespad=0, facecolor='w', edgecolor='k')

    a = fig.add_subplot(5, 1, 4)
    a.plot(df_output['洋室(小)'], linewidth = 1.0, label = 'calc-洋室(小)')
    a.plot(df_measure['洋室西'], linewidth = 1.0, label = '洋室西')
    a.legend(ncol=5, bbox_to_anchor=(0, 1.05, 1, 0), loc='lower left', borderaxespad=0, facecolor='w', edgecolor='k')

    a = fig.add_subplot(5, 1, 5)
    a.plot(df_output['ホール'], linewidth = 1.0, label = 'calc-ホール')
    a.plot(df_measure['2Fホール'], linewidth = 1.0, label = '2Fホール')
    a.legend(ncol=5, bbox_to_anchor=(0, 1.05, 1, 0), loc='lower left', borderaxespad=0, facecolor='w', edgecolor='k')

calc4(5)

# 浦和２

In [ ]:
def calc5():
    from vtsim import ground as gnd
    import matplotlib.pyplot as plt

    df_i = vt.read_csv('input_cooling7gatu.csv')

    a_dash  = {'A': 0.62 / 3600, 'B': 1.45 / 3600}
    n_value = {'A': 1.298, 'B': 1.81}

    area1 = {'A': 36.76, 'B': 23.55}                                             #床下南、床下北 面積
    area2 = {'A': 28.15, 'B': 23.55}                                             #階間南、階間北 面積
    leng  = {'A': 18.20, 'B': 14.56}                                             #床下南、床下北 外周長さ 

    u_value = 1.862                                                              #夏期:1.862 冬季:2.068
    phi_value = {'A': 1.168, 'B': 1.164}

    ground_dict = {'rg': 1.77, **gnd.gs.ground_dict('case0')}

    t_in = 26.0                                                                 #夏期:26℃　冬期:22℃　制御

    input = {
        'index': df_i.index,
        'sn': {
            '建築ダクト':   {'t_flag': vt.SN_CALC},
            '床下A':        {'t_flag': vt.SN_CALC},
            '床下B':        {'t_flag': vt.SN_CALC},
            '階間南':       {'t_flag': vt.SN_CALC},
            '階間北':       {'t_flag': vt.SN_CALC},
            'LDK':          {'t_flag': vt.SN_CALC, 'capa': 3667.053},
            '洗面所':       {'t_flag': vt.SN_CALC, 'capa': 1108.469},
            '洋室小':       {'t_flag': vt.SN_CALC, 'capa': 1251.681},
            '洋室大':       {'t_flag': vt.SN_CALC, 'capa': 2048.968},
            'ホール・階段': {'t_flag': vt.SN_FIX,  'capa': 1649.422, 't':    t_in},
            '外部':         {'t_flag': vt.SN_FIX,                    't':    df_i['t-ex']},
            '地盤':         {'t_flag': vt.SN_FIX,                    't':    15.2},
            '日射':         {'t_flag': vt.SN_FIX,  'h_sr': df_i['h_sr']}
        },
        'vn': {
            '建築ダクト -> 床下A':         {'vol': 300 / 3600},
            '建築ダクト -> 床下B':         {'vol': 100 / 3600},
            '建築ダクト -> 階間南':        {'vol': 300 / 3600},
            '建築ダクト -> 階間北':        {'vol': 100 / 3600},
            '床下A -> LDK':                {'vol': 300 / 3600},
            '床下B -> 洗面所':             {'vol':  50 / 3600},
            '床下B -> ホール・階段':       {'vol':  50 / 3600},
            '階間南 -> 洋室小':            {'vol': 100 / 3600},
            '階間南 -> 洋室大':            {'vol': 200 / 3600},
            '階間北 -> 洋室大':            {'vol':  50 / 3600},
            '階間北 -> ホール・階段':      {'vol':  50 / 3600},
            'LDK -> ホール・階段':         {'vol': 300 / 3600},
            '洗面所 -> ホール・階段':      {'vol':  50 / 3600},
            '洋室小 -> ホール・階段':      {'vol': 100 / 3600},
            '洋室大 -> ホール・階段 : 1':  {'vol': 200 / 3600},
            '洋室大 -> ホール・階段 : 2':  {'vol':  50 / 3600},
            'ホール・階段 -> 建築ダクト':  {'vol': 800 / 3600}
        },
        'tn': {
            '床下A -> LDK':               {'cdtc': area1['A'] * u_value},
            '床下B -> 洗面所':            {'cdtc': 8.281 * u_value},
            '床下B -> ホール・階段':      {'cdtc': 14.65 * u_value},
            '階間南 -> 洋室小':           {'cdtc': 11.66 * u_value},
            '階間南 -> 洋室大':           {'cdtc': 16.56 * u_value},
            '階間北 -> 洋室大':           {'cdtc':  6.62 * u_value},
            '階間北 -> ホール・階段':     {'cdtc': 12.42 * u_value},
            '階間南 -> 外部':             {'cdtc': leng['A'] * 0.24 * 0.38},
            '階間北 -> 外部':             {'cdtc': leng['B'] * 0.24 * 0.38},
            '床下A -> 外部':              {'cdtc': leng['A'] * phi_value['A']},
            '床下B -> 外部':              {'cdtc': leng['B'] * phi_value['B']},
            'LDK -> 外部':                {'cdtc': 54.03},
            '洗面所 -> 外部':             {'cdtc':  7.31},
            '洋室大 -> 外部':             {'cdtc': 25.29},
            '洋室小 -> 外部':             {'cdtc':  8.32},
            'ホール・階段 -> 外部':       {'cdtc': 35.54},
            'LDK -> 地盤':                {'area': 10.010 * 3.64 + 1.82 * 0.91, **ground_dict},
            '洗面所 -> 地盤':             {'area':  1.82  * 2.73 + 1.82 * 1.82, **ground_dict},
            'ホール・階段 -> 地盤':       {'area':  6.37  * 2.73,               **ground_dict},
            'LDK -> 日射':                {'ms':    1.609},
            '洗面所 -> 日射':             {'ms':    0.232},
            '洋室大 -> 日射':             {'ms':    0.980},
            '洋室小 -> 日射':             {'ms':    0.485},
            'ホール・階段 -> 日射':       {'ms':    0.988},
            'ホール・階段 -> 建築ダクト': {'ac_mode': vt.AC_COOLING, 'pre_tmp': t_in}
        }
    }

    vt.write_json(input, 'tutorial_urawa_2.json')
    vt.run_calc(input)


    df_output = vt.df_t[8760::]

    fig = plt.figure(facecolor='w', figsize = (20,90))
    fig.subplots_adjust(wspace = -0.1, hspace=0.7)

    a = fig.add_subplot(11, 1, 1)
    a.plot(df_output['LDK'], linewidth = 1.0, label = 'calc-LDK')
    a.plot(df_output['洗面所'], linewidth = 1.0, label = 'calc-洗面所')
    a.plot(df_output['洋室大'], linewidth = 1.0, label = 'calc-洋室(大)')
    a.plot(df_output['洋室小'], linewidth = 1.0, label = 'calc-洋室(小)')
    a.plot(df_output['ホール・階段'], linewidth = 1.0, label = 'calc-ホール・階段')
    a.plot(df_output['建築ダクト'], linewidth = 1.0, label = '建築ダクト')
    a.legend(ncol=5, bbox_to_anchor=(0, 1.05, 1, 0), loc='lower left', borderaxespad=0, facecolor='w', edgecolor='k')

calc5()

# 三鷹粉塵濃度

In [ ]:
def calc6():
    df_i = vt.read_csv('input_data.csv')

    input = {
        'index': df_i.index,
        'sn': {
            '外部':       {'c_flag': vt.SN_FIX,  'c': df_i['c_0']},
            '空調室':     {'c_flag': vt.SN_CALC, 'c': df_i['c_1'], 'v':  13.66, 'beta': 1e-5},
            '建築ダクト': {'c_flag': vt.SN_CALC, 'c': df_i['c_2'], 'v':   6.00, 'beta': 1e-5},
            '寝室':       {'c_flag': vt.SN_CALC, 'c': df_i['c_3'], 'v':  40.25, 'beta': 1e-5},
            '子供部屋':   {'c_flag': vt.SN_CALC, 'c': df_i['c_4'], 'v': 126.49, 'beta': 1e-5},
            '和室':       {'c_flag': vt.SN_CALC, 'c': df_i['c_5'], 'v':  16.56, 'beta': 1e-5},
            '洗面所':     {'c_flag': vt.SN_CALC, 'c': df_i['c_6'], 'v':   9.94, 'beta': 1e-5},
            '吹抜':       {'c_flag': vt.SN_CALC, 'c': df_i['c_7'], 'v': 100.46, 'beta': 1e-5, 'm':df_i['m']}
        },
        'vn': {
            '外部 -> 空調室':         {'vol': df_i['v_0']  / 3600},
            '空調室 -> 建築ダクト':   {'vol': df_i['v_1']  / 3600, 'eta': df_i['k']},
            '建築ダクト ->寝室':      {'vol': df_i['v_2']  / 3600},
            '建築ダクト -> 子供部屋': {'vol': df_i['v_3']  / 3600},
            '建築ダクト -> 和室':     {'vol': df_i['v_4']  / 3600},
            '建築ダクト -> 洗面所':   {'vol': df_i['v_5']  / 3600},
            '建築ダクト -> 吹抜':     {'vol': df_i['v_6']  / 3600},
            '寝室 -> 吹抜':           {'vol': df_i['v_7']  / 3600},
            '子供部屋 -> 吹抜':       {'vol': df_i['v_8']  / 3600},
            '和室 -> 吹抜':           {'vol': df_i['v_9']  / 3600},
            '洗面所 -> 外部':         {'vol': df_i['v_10'] / 3600},
            '洗面所 -> 吹抜':         {'vol': df_i['v_11'] / 3600},
            '吹抜 -> 外部':           {'vol': df_i['v_12'] / 3600},
            '吹抜 ->空調室':          {'vol': df_i['v_13'] / 3600}
        }
    }

    vt.write_json(input, 'tutorial_mitaka.json')
    vt.run_calc(input)

calc6()

# 床下空調

In [ ]:
def calc7():
    import pandas as pd

    df_i0 = vt.read_csv('input_data0.csv')
    df_i1 = vt.read_csv('input_data1_2.csv')
    df_i = pd.concat([df_i0, df_i1], axis = 1).interpolate('linear')

    VOL              = df_i['v1'] * ((0.2 / 2) ** 2 * 3.14 * 2)
    TEMP_UNDER_FLOOR = (df_i['t_in'] - df_i['t_ex']) * 0.7 + df_i['t_ex']

    input = {
        'index' : df_i.index,
        'sn': {
            '居室':       {'t_flag': vt.SN_CALC,  't': 22.0 , 'capa': 5000 * 1000},
            'ダクト内':   {'t_flag': vt.SN_FIX,   't': df_i['t_supply']},
            '床下（前）': {'t_flag': vt.SN_CALC},
            'パネル内':   {'t_flag': vt.SN_CALC},
            '床下（後）': {'t_flag': vt.SN_CALC},
            '床下':       {'t_flag': vt.SN_FIX,   't':    TEMP_UNDER_FLOOR},
            '外気':       {'t_flag': vt.SN_FIX,   't':    df_i['t_ex']},
            '日射':       {'t_flag': vt.SN_FIX,   'h_sr': df_i['h_sr']}
        },
        'vn': {
            '居室       -> ダクト内':    {'vol':  VOL},
            'ダクト内   -> 床下（前）':  {'vol':  VOL},
            '床下（前） -> パネル内':    {'vol':  VOL},
            'パネル内   -> 床下（後）':  {'vol':  VOL},
            '床下（後） -> 居室':        {'vol':  VOL}
        },
        'tn': {
            '居室       -> 外気': {'cdtc': 219.53 * 0.5},
            '床下（前） -> 床下': {'cdtc': 8.18},
            '床下（後） -> 床下': {'cdtc': 8.18},
            '居室       -> 日射': {'ms':   2.0}
        }
    }

    vt.write_json(input, 'tutorial_yukarela.json')
    vt.run_calc(input)

calc7()